## CSC427 Quantum Day One

_Date: 26 april 2022_

_Author: Burton Rosenberg_

### Getting ready. 

You will need the [qiskit package](https://qiskit.org/documentation/install.html). Qiskit is installed with pip, rather than conda.


__Notes for 2022:__ 

<pre>
conda create -n quantum
conda activate quantum
conda install pip
pip install qiskit
conda install jupyter
# perhaps
pip install qiskit-terra[visualization]
</pre>

This installation uses environments. List the environments that you have created
using the command,

- conda info --envs



In [1]:
import qiskit
import time, math

from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.jobstatus import JOB_FINAL_STATES, JobStatus
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer

qiskit.__qiskit_version__

#initialization
import matplotlib.pyplot as plt
import numpy as np

# what is this?
from qiskit.providers.ibmq import least_busy

args_g = []

# your api token from IBM, first time run.
# after that None is good

#api_token = 'abcdefghijklmnopqrstuvwxyz'
api_token = None 

def load_or_save_IBMQ_account(api_token=None):
    global args_g
    print('getting provider...')
    if api_token:
        # only needs to be done once
        # then is stored in e.g. ~/.qistkit/qiskitrc
        IBMQ.save_account(api_token)
    provider = IBMQ.load_account()
    return provider

def list_backends(provider):
    global args_g
    print('backends available ...')
    backends = provider.backends()
    for be in backends:
        st = be.status()
        if st.operational:
            print(f'\t{be.name()}, pending jobs:{st.pending_jobs}')

            
def run_quantum_circuit_on_backend(quantum_circuit,provider,backend):
    backend = provider.get_backend(backend)
    qobj = assemble(transpile(quantum_circuit, backend=backend), backend=backend)
    job = backend.run(qobj)
    return job


def wait_for_job(backend, job, wait_interval=5):
    backend = provider.get_backend(backend)
    retrieved_job = backend.retrieve_job(job.job_id())
    start_time = time.time()
    job_status = job.status()
    while job_status not in JOB_FINAL_STATES:
        print(f'Status @ {time.time() - start_time:0.0f} s: {job_status.name},'
              f' est. queue position: {job.queue_position()}')
        time.sleep(wait_interval)
        job_status = job.status()


provider = load_or_save_IBMQ_account(api_token)
list_backends(provider)

# choose your backend

#backend = 'ibmq_qasm_simulator'
#backend = 'ibmq_armonk'
#backend = 'ibmq_vigo'
#backend = 'ibmq_london'
backend = 'ibmq_lima'

# and so forth ... chose from the results given by provider.backends()

getting provider...
backends available ...
	ibmq_qasm_simulator, pending jobs:2
	ibmq_armonk, pending jobs:6
	ibmq_santiago, pending jobs:51
	ibmq_bogota, pending jobs:74
	ibmq_lima, pending jobs:1
	ibmq_belem, pending jobs:5
	ibmq_quito, pending jobs:13
	simulator_statevector, pending jobs:2
	simulator_mps, pending jobs:2
	simulator_extended_stabilizer, pending jobs:2
	simulator_stabilizer, pending jobs:2
	ibmq_manila, pending jobs:29


In [2]:
qc = QuantumCircuit(1)
qc.x(0)
qc.measure_all()
job = qiskit.execute(qc, qiskit.BasicAer.get_backend('qasm_simulator'), shots=1000)
print(qc.draw())

print(f'input, output, frequency\n\t|0> {job.result().get_counts()}')
qc = QuantumCircuit(1)
qc.x(0)
qc.x(0)
qc.measure_all()
job = qiskit.execute(qc, qiskit.BasicAer.get_backend('qasm_simulator'), shots=1000)
print(f'\t|1> {job.result().get_counts()}')

        ┌───┐ ░ ┌─┐
     q: ┤ X ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 
input, output, frequency
	|0> {'1': 1000}
	|1> {'0': 1000}


Although the frequencey of |+> and |-> are the same, they are not the same. 

In [3]:
qc = QuantumCircuit(1)
qc.h(0)
qc.measure_all()
job = qiskit.execute(qc, qiskit.BasicAer.get_backend('qasm_simulator'), shots=1000)
print(qc.draw())

print(f'input, output, frequency\n\t|0> {job.result().get_counts()}')
qc = QuantumCircuit(1)
qc.x(0)
qc.h(0)
qc.measure_all()
job = qiskit.execute(qc, qiskit.BasicAer.get_backend('qasm_simulator'), shots=1000)
print(f'\t|1> {job.result().get_counts()}')

print()
print('------------------------------')
print()

qc = QuantumCircuit(1)
qc.h(0)
qc.h(0)
qc.measure_all()
job = qiskit.execute(qc, qiskit.BasicAer.get_backend('qasm_simulator'), shots=1000)
print(qc.draw())

print(f'input, output, frequency\n\t|0> {job.result().get_counts()}')
qc = QuantumCircuit(1)
qc.x(0)
qc.h(0)
qc.h(0)
qc.measure_all()
job = qiskit.execute(qc, qiskit.BasicAer.get_backend('qasm_simulator'), shots=1000)
print(f'\t|1> {job.result().get_counts()}')

        ┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 
input, output, frequency
	|0> {'1': 492, '0': 508}
	|1> {'0': 503, '1': 497}

------------------------------

        ┌───┐┌───┐ ░ ┌─┐
     q: ┤ H ├┤ H ├─░─┤M├
        └───┘└───┘ ░ └╥┘
meas: 1/══════════════╩═
                      0 
input, output, frequency
	|0> {'0': 1000}
	|1> {'1': 1000}


In [4]:
# HXH show that the X doesn't matter. for  X H |1>  there will be a change of phase, but it wont matter

backend = 'ibmq_belem'

qhxh = QuantumCircuit(1)
qhxh.h(0)
qhxh.x(0)
qhxh.h(0)
qhxh.measure_all()
print(qhxh.draw())
job = qiskit.execute(qhxh, qiskit.BasicAer.get_backend('qasm_simulator'), shots=1000)
print(f'\t|0> {job.result().get_counts()}')
qhxh = QuantumCircuit(1)
qhxh.x(0)
qhxh.h(0)
qhxh.x(0)
qhxh.h(0)
qhxh.measure_all()
job = qiskit.execute(qhxh, qiskit.BasicAer.get_backend('qasm_simulator'), shots=1000)
print(f'\t|1> {job.result().get_counts()}')


        ┌───┐┌───┐┌───┐ ░ ┌─┐
     q: ┤ H ├┤ X ├┤ H ├─░─┤M├
        └───┘└───┘└───┘ ░ └╥┘
meas: 1/═══════════════════╩═
                           0 
	|0> {'0': 1000}
	|1> {'1': 1000}


### Tensor product of Hadamards

Creating a superposition of all states.


In [ ]:
backend = 'ibmq_lima'
#backend = 'ibmq_qasm_simulator'

qp = QuantumCircuit(2)
qp.h(0)
qp.h(1)
qp.measure_all()
print(qp.draw())

print(f'results: waiting for results from backend {backend} ...')
job = run_quantum_circuit_on_backend(qp,provider,backend)
wait_for_job(backend, job)
result = job.result()
print(f'results: {result.get_counts()}')


qp = QuantumCircuit(2)
qp.h(0)
qp.cx(0,1)
qp.measure_all()
print(qp.draw())

print(f'results: waiting for results from backend {backend} ...')
job = run_quantum_circuit_on_backend(qp,provider,backend)
wait_for_job(backend, job)
result = job.result()
print(f'results: {result.get_counts()}')

qp = QuantumCircuit(2)
qp.h(0)
qp.x(0)
qp.cx(0,1)
qp.measure_all()
print(qp.draw())

print(f'results: waiting for results from backend {backend} ...')
job = run_quantum_circuit_on_backend(qp,provider,backend)
wait_for_job(backend, job)
result = job.result()
print(f'results: {result.get_counts()}')


qp = QuantumCircuit(2)
qp.h(0)
qp.cx(0,1)
qp.x(0)
qp.measure_all()
print(qp.draw())

print(f'results: waiting for results from backend {backend} ...')
job = run_quantum_circuit_on_backend(qp,provider,backend)
wait_for_job(backend, job)
result = job.result()
print(f'results: {result.get_counts()}')


        ┌───┐ ░ ┌─┐   
   q_0: ┤ H ├─░─┤M├───
        ├───┤ ░ └╥┘┌─┐
   q_1: ┤ H ├─░──╫─┤M├
        └───┘ ░  ║ └╥┘
meas: 2/═════════╩══╩═
                 0  1 
results: waiting for results from backend ibmq_lima ...


/var/folders/x7/99kj7q8d23966f07qr0q71bm0000gn/T/ipykernel_6327/138421145.py:51: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  job = backend.run(qobj)


Status @ 0 s: VALIDATING, est. queue position: None


In [ ]:
backend = 'ibmq_qasm_simulator'

qp = QuantumCircuit(2)
qp.h(0)
qp.x(1)
qp.cx(0,1)

qp.measure_all()
print(qp.draw(output='mpl'))

print(f'results: waiting for results from backend {backend} ...')
job = run_quantum_circuit_on_backend(qp,provider,backend)
wait_for_job(backend, job)
result = job.result()
print(f'results: {result.get_counts()}')


### Greenberger–Horne–Zeilinger state

See https://en.wikipedia.org/wiki/Greenberger%E2%80%93Horne%E2%80%93Zeilinger_state

In [ ]:
#

def ghz():
    """
    Create a GHZ (entangled 3-qubit system) quantum circuit.
    from: Kory Becker primaryobjects  github
    """
    qc = QuantumCircuit(3)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(0, 2)
    qc.measure_all()
    return qc

    
    
qc = ghz()
print(qc.draw())
job = qiskit.execute(qc, qiskit.BasicAer.get_backend('qasm_simulator'), shots=1000)
print(job.result().get_counts())
plot_histogram(job.result().get_counts())